In [12]:
import requests
from dotenv import load_dotenv
import os
from mistralai.client import MistralClient
from pipelines.poker_dataset.struct_to_format_llm import struct_to_format_llm
from mistralai.models.chat_completion import ChatMessage

load_dotenv()


BACKEND_URL = os.getenv("BACKEND_URL") or "http://127.0.0.1:8000"

mistral_api_key = os.getenv("MISTRAL_API_KEY")
mistral_job_id = os.getenv("MISTRAL_JOB_ID")

client = MistralClient(api_key=mistral_api_key)
job = client.jobs.retrieve(mistral_job_id)

SETTINGS = {
    "client": client,
    "job": job
}

SETTINGS


{'client': <mistralai.client.MistralClient at 0x1ca29410670>,
 'job': DetailedJob(id='2cad6ef8-34b8-4e20-9431-db9ccccb0f1d', hyperparameters=TrainingParameters(training_steps=180, learning_rate=0.0001), fine_tuned_model='ft:open-mistral-7b:c6d4dfa8:20240626:2cad6ef8', model='open-mistral-7b', status='SUCCESS', job_type='FT', created_at=1719433586, modified_at=1719434336, training_files=['5cd53dbf-4f32-45e6-9b15-0c69b05b2cc6'], validation_files=['ff6b09ef-e7ba-4c1f-8a25-a35681ef8c93'], object='job', integrations=[], events=[Event(name='status-updated', data={'status': 'SUCCESS'}, created_at=1719434336), Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719433587), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719433586)], checkpoints=[Checkpoint(metrics=Metric(train_loss=0.0853, valid_loss=0.094045, valid_mean_token_accuracy=1.067359), step_number=180, created_at=1719434303), Checkpoint(metrics=Metric(train_loss=0.091283, valid_loss=0.094337, val

In [13]:
def play(card1, card2, position, mistral_settings=None):
    ''' Returns LLM's action given cards in hands and position at the table (UTG or BB)'''
    if position == 'UTG' :
        data = {   
        "variant" : "NT",
        "game_id" : 779459871,
        "hand_nb" : 0,
        "small_blind" : 0.25,
        "big_blind" : 0.50,
        "min_bet" : 0.25,
 
        "players" : ["n0hvn", "tbmfc", "naprimer", "Log_in", "IlxxxlI", "gmjohn", "MANTISGUYV10", "BiGFck"],
        "starting_stacks" : [55.50, 28.47, 55.31, 15.15, 20, 28.76, 57.49, 17],
        "players_seats" : [1, 2, 3, 4, 5, 7, 8, 9],
 
        "button_seat" : 2,
        "player_small_blind" : "naprimer",
        "player_big_blind" : "Log_in",
 
        "player" : "IlxxxlI",
        "cards_player" : [card1, card2],
        "current_street" : "pre_flop",
 
        "dealed_cards" : {
                    "flop": [],
                    "turn": [],
                    "river": []
                   },
 
 
        "actions" : {"pre_flop" : {"players": [],
                            "actions": [],
                            "value": []},
                "post_flop" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_turn" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_river" : {
                            "players": [],
                            "actions": [],
                            "value": []
                                }
               },
 
 
        "winners" : [],
        "finishing_stacks": [],
        "card_shown_by_players" : []
        }

    elif position == 'BB':
        data = {   "variant" : "NT",
        "game_id" : 779460276,
        "hand_nb" : 0,
 
        "small_blind" : 0.25,
        "big_blind" : 0.50,
        "min_bet" : 0.25,
 
        "players" : ["n0hvn", "tbmfc", "naprimer", "Log_in", "IlxxxlI", "gmjohn", "MANTISGUYV10", "BiGFck"],
        "starting_stacks" : [55.50, 28.47, 55.06, 14.65, 20, 28.76, 58.24, 17],
        "players_seats" : [1, 2, 3, 4, 5, 7, 8, 9],
 
        "button_seat" : 3,
        "player_small_blind" : "Log_in",
        "player_big_blind" : "IlxxxlI",
 
        "player" : "IlxxxlI",
        "cards_player" : [card1, card2],
        "current_street" : "pre_flop",
 
        "dealed_cards" : {
                    "flop": [],
                    "turn": [],
                    "river": []
                   },
 
 
        "actions" : {"pre_flop" : {"players": ["gmjohn", "MANTISGUYV10", "BiGFck", "n0hvn", "tbmfc", "naprimer", "Log_in"],
                            "actions": ["f", "cc", "f", "f", "f", "f", "f"],
                            "value": [None, None, None, None, None, None, None]},
                "post_flop" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_turn" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_river" : {
                            "players": [],
                            "actions": [],
                            "value": []
                                }
               },
 "response"
 
        "winners" : [],
        "finishing_stacks": [],
        "card_shown_by_players" : []
        }
    
    #Envoie l'inférence dans le back  action = inference(data, argument_pour_inference)
    #Envoie le json data et reçoit l'action
    
    if not mistral_settings:
        route = "/predict_test"
        route = "/predict"
        response = requests.post(f'{BACKEND_URL}{route}', json=data)
        response_json = response.json()

        if 'response' in response_json:
            action = response_json['response']
            return action[0]
        else:
            raise ValueError(f"Error in response: {response_json}")
        
    else:
        hand_format_llm = struct_to_format_llm(data)
        chat_response = client.chat(
                model=job.fine_tuned_model,
                messages=[ChatMessage(role="user", content=hand_format_llm)],
            )
        action = chat_response.choices[0].message.content
        return action
        



In [14]:
card1 = "2s"
card2 = "2h"
position = "UTG"
mistral_settings = SETTINGS
action = play(card1, card2, position, mistral_settings)
action

'folds'

In [15]:
import numpy as np
import random

def range_hands():
    '''Generate hands for all possible pairs (suited and off-suited)'''
    range_matrix = np.zeros((13, 13), dtype=object)
    suits = ["s", "h", "d", "c"]
    values = ["A", "K", "Q", "J", "10", "9", "8", "7", "6", "5", "4", "3", "2"]
    
    for i in range(13):
        for j in range(13):
            if j > i:  # Suited hands
                color = random.choice(suits)
                card1, card2 = values[i] + color, values[j] + color
                range_matrix[i][j] = card1, card2
            else:  # Off-suited hands
                colors = random.sample(suits, 2)
                card1, card2 = values[i] + colors[0], values[j] + colors[1]
                range_matrix[i][j] = card1, card2

    return range_matrix

# Example
print(range_hands())


[[('Ad', 'Ac') ('Ah', 'Kh') ('Ad', 'Qd') ('As', 'Js') ('Ad', '10d')
  ('Ac', '9c') ('Ah', '8h') ('Ac', '7c') ('Ad', '6d') ('Ad', '5d')
  ('Ad', '4d') ('Ac', '3c') ('Ah', '2h')]
 [('Kh', 'Ac') ('Kh', 'Kd') ('Kd', 'Qd') ('Kh', 'Jh') ('Ks', '10s')
  ('Kh', '9h') ('Kc', '8c') ('Ks', '7s') ('Ks', '6s') ('Kh', '5h')
  ('Kd', '4d') ('Ks', '3s') ('Kc', '2c')]
 [('Qh', 'Ac') ('Qd', 'Ks') ('Qh', 'Qs') ('Qh', 'Jh') ('Qd', '10d')
  ('Qs', '9s') ('Qs', '8s') ('Qd', '7d') ('Qc', '6c') ('Qh', '5h')
  ('Qc', '4c') ('Qd', '3d') ('Qs', '2s')]
 [('Jc', 'Ah') ('Js', 'Kh') ('Jc', 'Qh') ('Js', 'Jh') ('Jh', '10h')
  ('Jh', '9h') ('Jh', '8h') ('Jd', '7d') ('Jc', '6c') ('Js', '5s')
  ('Js', '4s') ('Js', '3s') ('Js', '2s')]
 [('10d', 'As') ('10c', 'Kh') ('10d', 'Qc') ('10s', 'Jc') ('10d', '10c')
  ('10d', '9d') ('10s', '8s') ('10h', '7h') ('10d', '6d') ('10d', '5d')
  ('10c', '4c') ('10h', '3h') ('10h', '2h')]
 [('9h', 'Ad') ('9d', 'Ks') ('9h', 'Qd') ('9c', 'Js') ('9h', '10d')
  ('9h', '9c') ('9d', '8d') ('9c',

In [16]:
def model_range_generator(position, mistral_settings):
    """Fulfill a range with LLM's actions"""
    range_matrix = range_hands()
    for i in range(13):
        for j in range(13):
            card1, card2 = range_matrix[i][j]
            action = play(card1, card2, position, mistral_settings)
            range_matrix[i][j] = action
            print(f"Action for {card1} {card2} is {action}")

    return range_matrix

In [19]:
range_matrix_bb = model_range_generator(position='BB', mistral_settings=SETTINGS)

Action for Ad Ah is raises (1.75)
Action for Ah Kh is raises (1.25)
Action for Ac Qc is raises (1.25)
Action for Ad Jd is raises (1.75)
Action for Ac 10c is raises (1.25)
Action for Ac 9c is checks
Action for Ad 8d is checks
Action for Ah 7h is raises (1.25)
Action for Ac 6c is checks
Action for Ah 5h is raises (1.25)
Action for As 4s is checks
Action for Ad 3d is checks
Action for Ac 2c is checks
Action for Kc As is raises (1.25)
Action for Ks Kh is raises (1.50)
Action for Ks Qs is raises (1)
Action for Kc Jc is raises (1.25)
Action for Kc 10c is raises (1.25)
Action for Kh 9h is raises (1.25)
Action for Kd 8d is checks
Action for Ks 7s is raises (1)
Action for Kc 6c is checks
Action for Kd 5d is checks
Action for Kc 4c is checks
Action for Kd 3d is checks
Action for Ks 2s is checks
Action for Qh Ac is raises (1.25)
Action for Qc Kh is raises (1.25)
Action for Qh Qd is raises (1.75)
Action for Qc Jc is raises (1)
Action for Qh 10h is raises (1.25)
Action for Qd 9d is raises (1.15)
Ac

MistralException: Unexpected exception (RemoteProtocolError): Server disconnected without sending a response.

In [ ]:
#save the range
import pickle

folder = "./../../data/4_results/range_matrix/"
name = "range_matrix_bb"
id = "001"
with open(f"{folder}{name}_{id}.pkl", "wb") as f:
    pickle.dump(range_matrix_bb, f)

# save the hand format
data = {   "variant" : "NT",
        "game_id" : 779460276,
        "hand_nb" : 0,
 
        "small_blind" : 0.25,
        "big_blind" : 0.50,
        "min_bet" : 0.25,
 
        "players" : ["n0hvn", "tbmfc", "naprimer", "Log_in", "IlxxxlI", "gmjohn", "MANTISGUYV10", "BiGFck"],
        "starting_stacks" : [55.50, 28.47, 55.06, 14.65, 20, 28.76, 58.24, 17],
        "players_seats" : [1, 2, 3, 4, 5, 7, 8, 9],
 
        "button_seat" : 3,
        "player_small_blind" : "Log_in",
        "player_big_blind" : "IlxxxlI",
 
        "player" : "IlxxxlI",
        "cards_player" : [card1, card2],
        "current_street" : "pre_flop",
 
        "dealed_cards" : {
                    "flop": [],
                    "turn": [],
                    "river": []
                   },
 
 
        "actions" : {"pre_flop" : {"players": ["gmjohn", "MANTISGUYV10", "BiGFck", "n0hvn", "tbmfc", "naprimer", "Log_in"],
                            "actions": ["f", "cc", "f", "f", "f", "f", "f"],
                            "value": [None, None, None, None, None, None, None]},
                "post_flop" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_turn" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_river" : {
                            "players": [],
                            "actions": [],
                            "value": []
                                }
               },
 "response"
 
        "winners" : [],
        "finishing_stacks": [],
        "card_shown_by_players" : []
        }
    
with open(f"{folder}{name}_{id}_data_struct.pkl", "wb") as f:
    pickle.dump(data, f)

# save the hand format llm
hand_format_llm = struct_to_format_llm(data)
with open(f"{folder}{name}_{id}_hand_format_llm.pkl", "wb") as f:
    pickle.dump(hand_format_llm, f)
    

In [20]:
range_matrix_utg = model_range_generator(position='UTG', mistral_settings=SETTINGS)

In [21]:
#save the range
import pickle

folder = "./../../data/4_results/range_matrix/"
name = "range_matrix_utg"
id = "001"

with open(f"{folder}{name}_{id}.pkl", "wb") as f:
    pickle.dump(range_matrix_utg, f)



data = {   
        "variant" : "NT",
        "game_id" : 779459871,
        "hand_nb" : 0,
        "small_blind" : 0.25,
        "big_blind" : 0.50,
        "min_bet" : 0.25,
 
        "players" : ["n0hvn", "tbmfc", "naprimer", "Log_in", "IlxxxlI", "gmjohn", "MANTISGUYV10", "BiGFck"],
        "starting_stacks" : [55.50, 28.47, 55.31, 15.15, 20, 28.76, 57.49, 17],
        "players_seats" : [1, 2, 3, 4, 5, 7, 8, 9],
 
        "button_seat" : 2,
        "player_small_blind" : "naprimer",
        "player_big_blind" : "Log_in",
 
        "player" : "IlxxxlI",
        "cards_player" : ["CARD1", "CARD2"],
        "current_street" : "pre_flop",
 
        "dealed_cards" : {
                    "flop": [],
                    "turn": [],
                    "river": []
                   },
 
 
        "actions" : {"pre_flop" : {"players": [],
                            "actions": [],
                            "value": []},
                "post_flop" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_turn" : {
                            "players": [],
                            "actions": [],
                            "value": []},
                "post_river" : {
                            "players": [],
                            "actions": [],
                            "value": []
                                }
               },
 
 
        "winners" : [],
        "finishing_stacks": [],
        "card_shown_by_players" : []
        }




array([['raises (1.50)', 'raises (1.25)', 'raises (1.50)',
        'raises (1.50)', 'raises (1.25)', 'folds', 'folds', 'folds',
        'folds', 'folds', 'folds', 'folds', 'folds'],
       ['raises (1.50)', 'raises (1.50)', 'raises (1.50)', 'folds',
        'raises (1.25)', 'folds', 'folds', 'folds', 'folds', 'folds',
        'folds', 'folds', 'folds'],
       ['raises (1.50)', 'folds', 'raises (1.75)', 'folds', 'folds',
        'folds', 'folds', 'folds', 'folds', 'folds', 'folds', 'folds',
        'folds'],
       ['raises (1.25)', 'folds', 'folds', 'raises (1.50)', 'folds',
        'folds', 'folds', 'folds', 'folds', 'folds', 'folds', 'folds',
        'folds'],
       ['raises (1.50)', 'folds', 'folds', 'folds', 'raises (1.50)',
        'folds', 'folds', 'folds', 'folds', 'folds', 'folds', 'folds',
        'folds'],
       ['folds', 'folds', 'folds', 'folds', 'folds', 'raises (1.50)',
        'folds', 'folds', 'folds', 'folds', 'folds', 'folds', 'folds'],
       ['folds', 'folds', 'f

In [ ]:
def range_accuracy(range1, range2):
    '''Calcule accuracy of range1 according to a reference range range2'''
    matching_actions = 0
    for i in range(13):
        for j in range(13):
            if range1[i][j] == range2[i][j] :
                matching_actions += 1
    return matching_actions/(13*13)